In [7]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

In [27]:
driver = webdriver.Chrome()

In [28]:
driver.get("https://www.amazon.in/s?k=sunglass&crid=389J7CS8RFIK9&sprefix=sungla%2Caps%2C356&ref=nb_sb_noss_2")

In [29]:
html_data = BeautifulSoup(driver.page_source,"html.parser")

In [30]:
no_of_pages = int(html_data.find("span",{'class':'s-pagination-item s-pagination-disabled'}).text)

In [31]:
no_of_pages

7

In [33]:
products = html_data.find_all("div",{'data-component-type':'s-search-result'})

In [34]:
titles = []
images = []
prices = []
ratings = []
urls = []

for i in tqdm(range(no_of_pages)):
    url = "https://www.amazon.in/s?k=sunglass&crid=389J7CS8RFIK9&sprefix=sungla%2Caps%2C356&ref=nb_sb_noss_2"+"&page="+str(i+1)
    driver.get(url)
    time.sleep(3)
    html_data = BeautifulSoup(driver.page_source,"html.parser")
    products = html_data.find_all("div",{'data-component-type':'s-search-result'})
    for product in products:
        title = product.h2.find("span").text
        titles.append(title)

        img = product.find("img",{'class':'s-image'})['src']
        images.append(img)

        rating = product.find('span',{'class':'a-icon-alt'})
        if rating is not None:
            rating = rating.text
            rating = float(rating[0:4])
        ratings.append(rating)

        price = product.find('span',{'class':'a-price-whole'})
        if price is not None:
            price = '₹'+price.text
        prices.append(price)

        product_link = 'https://www.amazon.in'+ product.find('a',{'class':'a-link-normal s-line-clamp-2 s-link-style a-text-normal'})['href']
        urls.append(product_link)

100%|██████████| 7/7 [00:54<00:00,  7.82s/it]


In [36]:
len(titles)

390

In [37]:
print(len(prices))

390


In [39]:
data = pd.DataFrame({
    'titles':titles,
    'images':images,
    'prices':prices,
    'ratings':ratings,
    'purls':urls
})

In [40]:
data

,titles,images,prices,ratings,purls
0,Fastrack,https://m.media-amazon.com/images/I/61xbGkFmwv...,₹989,4.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Fastrack,https://m.media-amazon.com/images/I/313a79CkfA...,₹759,4.0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Dervin,https://m.media-amazon.com/images/I/51hVf+IEFk...,₹426,4.6,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Fastrack,https://m.media-amazon.com/images/I/81GtbybTCo...,₹961,4.1,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,VINCENT CHASE EYEWEAR,https://m.media-amazon.com/images/I/41Dr9I4x1E...,₹529,4.1,https://www.amazon.in/Polarized-Protected-Sung...
...,...,...,...,...,...
385,grey jack,https://m.media-amazon.com/images/I/61mqLx8scC...,₹718,3.7,https://www.amazon.in/grey-jack-Semi-Rimless-S...
386,OPTICSFIT,https://m.media-amazon.com/images/I/41c75NXa50...,₹589,4.4,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
387,Voyage,https://m.media-amazon.com/images/I/51fNP30HB0...,₹655,4.1,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
388,OAKTER,https://m.media-amazon.com/images/I/61q+-zmDqd...,"₹2,499",4.2,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [41]:
data.to_csv('sunglass.csv',index=False)